In [234]:
import csv
import pandas as pd
import numpy as np
import sklearn

from pymongo import MongoClient

ALL_COLORS = ['W', 'U', 'B', 'R', 'G']

PATH = './data/draft_data_public.MKM.PremierDraft.csv'

NO_NO = 0
NO_YES = 1
YES_YES = 2

NO = 0
YES = 1

MAX_PIP_COUNT = 40
NUM_COLORS = 5
NUM_IN_PACK = 13
NUM_PACKS = 3

In [340]:
client = MongoClient()
cards_en = client.scryfall.cards_en

with open(PATH) as csvfile:
    draft_data = csv.reader(csvfile)
    COLUMNS = next(draft_data)

NAMES = [col.split('pack_card_')[1] for col in columns if 'pack_card_' in col]
PICK_IDX = columns.index('pick')
DRAFTID_IDX = columns.index('draft_id')
DRAFTTIME_IDX = columns.index('draft_time')
PACKNUM_IDX = columns.index('pack_number')
PICKNUM_IDX = columns.index('pick_number')
PACK_IDX = columns.index('pack_card_A Killer Among Us')
POOL_IDX = columns.index('pool_A Killer Among Us')
NUM_CARDS = len(names)


In [236]:
def get_card(card_name):
    card = cards_en.find_one({'name': card_name})

    if card is None:
        query = {            
            'card_faces': {
                '$elemMatch': {
                    'name': card_name
                }
            },
        }

        card = cards_en.find_one(query)

    if card is None:
        error_message = 'No match for card name {}'.format(card_name)
        print(error_message)

    return(card)

def card_vector(index, row):
    return np.array([int(x) for x in row[index:index + NUM_CARDS]])

In [238]:
alsa_filepath = './data/alsa.npy'
# calculate ALSA

# last_seen = np.zeros(len(names))
# counts = np.zeros(len(names))
# with open(PATH) as csvfile:
#     draft_data = csv.reader(csvfile)
#     next(draft_data)
#     draft_id = None
#     pack_num = None
#     row_last_seen = np.zeros(len(names))
    
#     for row in draft_data:
#         if draft_id != row[DRAFTID_IDX] or pack_num != row[PACKNUM_IDX]:
#             last_seen += row_last_seen
#             counts += np.where(row_last_seen, 1, 0)
#             draft_id = row[DRAFTID_IDX]
#             pack_num = int(row[PACKNUM_IDX])
#             row_last_seen = np.zeros(NUM_CARDS)
#         pack = card_vector(PACK_IDX, row)
#         np.putmask(row_last_seen, pack, int(row[PICKNUM_IDX]))
        
# alsa = last_seen / counts
# alsa_series = pd.Series(alsa, index=NAMES)
# alsa_series = alsa_series[pd.notna(alsa)]

# with open(alsa_filepath, 'wb') as f:
#     np.save(f, alsa)

alsa = np.load(alsa_filepath)

In [273]:
def pip_vector(card_name):
    card = get_card(card_name)
    if card is None:
        print(card_name + ' Not Found')
    mana_cost = card['mana_cost']
    return np.array([mana_cost.count(f'{{{color}}}') for color in ALL_COLORS])  \
        + 1/2 * np.array([mana_cost.count(f'/{color}') for color in ALL_COLORS])\
        + 1/2 * np.array([mana_cost.count(f'{color}/') for color in ALL_COLORS])

PIP_LOOKUP = np.array([pip_vector(name) for name in names])

def pool_pips(pool):
    return np.matmul(pool.reshape(1,NUM_CARDS), PIP_LOOKUP)[0]

def pick_number(row):
    return int(row[PICKNUM_IDX]) + NUM_IN_PACK * int(row[PACKNUM_IDX])

def pip_counts_to_matrix(pip_counts):
    matrix = np.zeros((NUM_COLORS, MAX_PIP_COUNT), dtype=np.intc)
    for i in range(NUM_COLORS):
        matrix[i][0:int(pip_counts[i])] = 1
    return matrix
       


In [508]:
COLOR_NAMES = ['White', 'Blue', 'Black', 'Red', 'Green']

def print_pack(row):
    print( "========" )
    print( f"Draft Id: {row[DRAFTID_IDX]}, Pack {row[PACKNUM_IDX]}, Pick {row[PICKNUM_IDX]}")
    print( "--------" )
    pack_vector = card_vector(PACK_IDX, row)
    if (len(pack_vector) > 1):
        pack_alsa = np.where(pack_vector, alsa, NUM_IN_PACK + 1)
        min_indices = np.argpartition(pack_alsa, 2)
        for idx in min_indices[0:2]:
            print( names[idx] + f" (ALSA: {pack_alsa[idx]}" )
        print( f"And {pack_vector.sum() - 2} other cards" )
    else:
        print( names[pack_vector.index(1)] )
    print("--------")
    print(f"Picked: {row[PICK_IDX]}")
    print("========")
    print("")
        
def print_pool_dist(pool_matrix):
    pips = pool_matrix.sum(axis=1)
    total = pips.sum()
    args = np.flip(pips.argsort())
    print("********")
    for color in args:
        print( ALL_COLORS[color]*round((pips[color]))  + f" ({pips[color]:.2f} {COLOR_NAMES[color]})")
    print(f'{total:.2f} total')
    print("********")
    print("")
            

In [512]:
experiment_counts = np.zeros((NUM_PACKS * NUM_IN_PACK, len([NO_NO, NO_YES, YES_YES]), NUM_COLORS, MAX_PIP_COUNT), dtype=np.intc)
unseen_events = np.zeros((NUM_PACKS * NUM_IN_PACK, len([NO_NO, NO_YES, YES_YES]), NUM_COLORS, MAX_PIP_COUNT), dtype=np.intc)
sender_events = np.zeros((NUM_PACKS * NUM_IN_PACK, NUM_CARDS, len([NO_NO, NO_YES, YES_YES]), NUM_COLORS, MAX_PIP_COUNT), dtype=np.intc)
receiver_events = np.zeros((NUM_PACKS * NUM_IN_PACK, NUM_CARDS, len([NO_NO, NO_YES, YES_YES]), NUM_COLORS, MAX_PIP_COUNT), dtype=np.intc) 
test_rows = []

with open(PATH) as csvfile:
    draft_data = csv.reader(csvfile)
    next(draft_data) # discard names
    row = next(draft_data)
    
    while(row[DRAFTTIME_IDX] < '2024-02-20'):
        row = next(draft_data)

    while(row[DRAFTTIME_IDX] < '2024-03-18'):
        pack = card_vector(PACK_IDX, row) 
        pack_alsa = np.where(pack, alsa, NUM_IN_PACK+1)
        pool = card_vector(POOL_IDX, row)
        pool_pip_counts_pre = pool_pips(pool)
        pool_pre_matrix = pip_counts_to_matrix(pool_pip_counts_pre)
        pool_pip_counts_post = pool_pip_counts_pre + PIP_LOOKUP[names.index(row[PICK_IDX])]

        experiment_counts[pick_number(row), NO_NO] += 1 - pool_pre_matrix
        experiment_counts[pick_number(row), NO_YES] += 1 - pool_pre_matrix
        experiment_counts[pick_number(row), YES_YES] += pool_pre_matrix

        pool_post_matrix = pip_counts_to_matrix(pool_pip_counts_post)
                
        no_no_matrix = 1 - pool_post_matrix
        no_yes_matrix = pool_post_matrix - pool_pre_matrix

        unseen_events[pick_number(row), NO_NO] += no_no_matrix
        unseen_events[pick_number(row), NO_YES] += no_yes_matrix
        unseen_events[pick_number(row), YES_YES] += pool_pre_matrix
        
        receiver_events[pick_number(row), min_indices[0], NO_NO] += no_no_matrix
        receiver_events[pick_number(row), min_indices[0], NO_YES] += no_yes_matrix
        receiver_events[pick_number(row), min_indices[0], YES_YES] += pool_pre_matrix
                
        min_indices = np.argpartition(pack_alsa, 2)
        top_passed_card = min_indices[1] if names[min_indices[0]] == row[columns.index('pick')] else min_indices[0]
            
        sender_events[pick_number(row), top_passed_card, NO_NO] += no_no_matrix
        sender_events[pick_number(row), top_passed_card, NO_YES] += no_yes_matrix
        sender_events[pick_number(row), top_passed_card, YES_YES] += pool_pre_matrix

        row = next(draft_data)

    test_rows.append(row)
    for row in draft_data:
        test_rows.append(row)

        

In [513]:
sender_parameters = np.nan_to_num(sender_events / experiment_counts[:, np.newaxis, ...], 0)
receiver_parameters = np.nan_to_num(receiver_events / experiment_counts[:, np.newaxis, ...], 0)
unseen_parameters = np.nan_to_num(unseen_events / experiment_counts, 0)

/tmp/ipykernel_29055/3405331065.py:1: RuntimeWarning: invalid value encountered in divide
  sender_parameters = np.nan_to_num(sender_events / experiment_counts[:, np.newaxis, ...], 0)
/tmp/ipykernel_29055/3405331065.py:2: RuntimeWarning: invalid value encountered in divide
  receiver_parameters = np.nan_to_num(receiver_events / experiment_counts[:, np.newaxis, ...], 0)
/tmp/ipykernel_29055/3405331065.py:3: RuntimeWarning: invalid value encountered in divide
  unseen_parameters = np.nan_to_num(unseen_events / experiment_counts, 0)


In [514]:
def update_sender(prior, pack_received, pack_num, pick_num):
    # pack_received is the remaining cards as an np mask vector after the pack/pick being modeled, e.g. 12 cards for p1p1
    
    if pick_num == 12:
        no_no = unseen_parameters[13*pack_num + pick_num, NO_NO]
        no_yes = unseen_parameters[13*pack_num + pick_num, NO_YES]
        yes_yes = unseen_parameters[13*pack_num + pick_num, YES_YES]
    else:
        pack_alsa = np.where(pack_received, alsa, NUM_IN_PACK+1)
        top_received_card = np.argmin(pack_alsa)
        
        no_no = sender_parameters[13*pack_num + pick_num, top_received_card, NO_NO]
        no_yes = sender_parameters[13*pack_num + pick_num, top_received_card, NO_YES]
        yes_yes = sender_parameters[13*pack_num + pick_num, top_received_card, YES_YES]

    # Bayesian updating for part of both sides that becomes "yes" to each distribution function
    yes_update = prior * yes_yes + (1-prior) * no_yes

    # Bayesian updating for the part that remains no
    no_update = (1 - prior) * no_no

    return yes_update / (yes_update + no_update)

def update_receiver(prior, pack_sent, pack_num, pick_num):
    # pack_sent is all the cards sent, empty for pxp1
    
    if pick_num == 0:
        no_no = unseen_parameters[13*pack_num + pick_num, NO_NO]
        no_yes = unseen_parameters[13*pack_num + pick_num, NO_YES]
        yes_yes = unseen_parameters[13*pack_num + pick_num, YES_YES]
    else:
        pack_alsa = np.where(pack_sent, alsa, NUM_IN_PACK + 1)
        top_sent_card = np.argmin(pack_alsa)
            
        yes_yes = receiver_parameters[13*pack_num + pick_num, top_sent_card, YES_YES]
        no_yes = receiver_parameters[13*pack_num + pick_num, top_sent_card, NO_YES]
        no_no = receiver_parameters[13*pack_num + pick_num, top_sent_card, NO_NO]

    # Bayesian updating for part of both sides that becomes "yes" to each distribution function
    yes_update = prior * yes_yes + (1-prior) * no_yes

    # Bayesian updating for the part that remains no
    no_update = (1 - prior) * no_no

    return yes_update / (yes_update + no_update)
    
    

In [522]:
def get_pick_num(row):
    return int(row[PICKNUM_IDX])

def get_pack_num(row):
    return int(row[PACKNUM_IDX])

LEFT = 'left'
RIGHT = 'right'

def update_as_if_sender(prior, row):
    pack = card_vector(PACK_IDX, row)
    pack[names.index(row[PICK_IDX])] -= 1
    return update_sender(prior, pack, get_pack_num(row), get_pick_num(row))

def update_as_if_receiver(prior, row):
    pack = card_vector(PACK_IDX, row)
    return update_receiver(prior, pack, get_pack_num(row), get_pick_num(row))

def simulate_full_pack_as_if_sender(prior, pack_rows, full_print=True):
    for row in pack_rows:
        if full_print:
            print('Updating distribution as if this pack was sent from the modeled user, less the picked card')
            print_pack(row)
        prior = update_as_if_sender(prior, row)
        if full_print:
            print_pool_dist(prior)
    return prior

def simulate_full_pack_as_if_receiver(prior, pack_rows, full_print=True):
    for row in pack_rows:
        if full_print:
            print('Updating distribution as if this pack was received by the modeled user from your seat (or opened)')
            print_pack(row)
        prior = update_as_if_receiver(prior, row)
        if full_print:
            print_pool_dist(prior)
    return prior

def simulate_on_test_draft(draft_rows, as_seat, full_print=True):
    assert len(draft_rows) == 3 * NUM_IN_PACK
    prior = np.zeros((5,40))

    if as_seat == LEFT:
        prior = simulate_full_pack_as_if_sender(prior, draft_rows[0 : NUM_IN_PACK], full_print)
        prior = simulate_full_pack_as_if_receiver(prior, draft_rows[NUM_IN_PACK : 2 * NUM_IN_PACK], full_print)
        prior = simulate_full_pack_as_if_sender(prior, draft_rows[2 * NUM_IN_PACK : 3 * NUM_IN_PACK], full_print)
    else:
        prior = simulate_full_pack_as_if_receiver(prior, draft_rows[0 : NUM_IN_PACK], full_print)
        prior = simulate_full_pack_as_if_sender(prior, draft_rows[NUM_IN_PACK : 2 * NUM_IN_PACK], full_print)
        prior = simulate_full_pack_as_if_receiver(prior, draft_rows[2 * NUM_IN_PACK : 3 * NUM_IN_PACK], full_print)
    if full_print:
        row = draft_rows[-1]
        print('Actual final pool:')
        print_pool_dist(pip_counts_to_matrix(pool_pips(card_vector(POOL_IDX, row))).astype(np.double))
       
    return prior

In [527]:
test_draft = test_rows[2 * 39:3* 39]
simulate_on_test_draft(test_draft, RIGHT)

Updating distribution as if this pack was received by the modeled user from your seat (or opened)
Draft Id: d8daa80f9abf486980be53f8c70d08b9, Pack 0, Pick 0
--------
Warleader's Call (ALSA: 1.8971464408905614
Marketwatch Phantom (ALSA: 3.1261844248941024
And 11 other cards
--------
Picked: Warleader's Call

********
 (0.43 White)
 (0.33 Red)
 (0.31 Green)
 (0.28 Blue)
 (0.22 Black)
1.57 total
********

Updating distribution as if this pack was received by the modeled user from your seat (or opened)
Draft Id: d8daa80f9abf486980be53f8c70d08b9, Pack 0, Pick 1
--------
Inside Source (ALSA: 3.0498631302355257
Murder (ALSA: 3.4291306775653743
And 10 other cards
--------
Picked: Inside Source

********
W (0.94 White)
 (0.50 Green)
 (0.45 Red)
 (0.43 Blue)
 (0.32 Black)
2.64 total
********

Updating distribution as if this pack was received by the modeled user from your seat (or opened)
Draft Id: d8daa80f9abf486980be53f8c70d08b9, Pack 0, Pick 2
--------
Commercial District (ALSA: 3.09460348229

array([[9.52143657e-01, 9.25221581e-01, 8.84188512e-01, 8.14094988e-01,
        7.46819376e-01, 7.07306608e-01, 5.98336598e-01, 6.70668010e-01,
        6.18645531e-01, 5.91050794e-01, 5.72432282e-01, 5.71047298e-01,
        5.93081027e-01, 5.81606747e-01, 4.97682943e-01, 4.95866752e-01,
        4.38127770e-01, 3.91264055e-01, 3.23811416e-01, 2.80657280e-01,
        2.16126443e-01, 1.46781883e-01, 1.52823617e-01, 4.70609660e-02,
        3.29879186e-02, 6.69533878e-02, 3.33231025e-02, 2.89364881e-02,
        4.94481283e-03, 8.53970965e-04, 1.70502984e-03, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [9.92707816e-01, 9.59521502e-01, 9.14467917e-01, 8.66379382e-01,
        7.57367275e-01, 7.39032537e-01, 7.02767224e-01, 5.49320883e-01,
        4.46497127e-01, 3.84515217e-01, 3.88100178e-01, 3.65370508e-01,
        3.12739055e-01, 2.44757913e-01, 2.48269970e-01, 1.75958